In [57]:
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle
import time
import tkinter

In [58]:
# Kartenfarbe: 0 = Karo; 1 = Herz; 2 = Pik; 3 = Kreuz
# Kartenbezeichnung: 0 = 7; 1 = 8; 2 = 9; 3 = 10; 4 = Bube, 5 = Dame; 6 = König, 7 = Ass
card_values = [7, 8, 9, 10, 10, 10, 10, 11]
CorX=[290, 420, 550, 290, 420, 550, 30, 160, 290, 550, 680, 810]
CorY=[420, 420, 420, 220, 220, 220, 20, 20, 20, 20, 20, 20]

In [59]:
deck= []
for i in range(4):
    for j in range(8):
        deck.append((i,j))
shuffle(deck)

In [60]:
hand = deck[:3]
oponent_Hand_1 = deck[3:6]
oponent_hand_2 = deck[6:9]
open_cards = deck[9:12]
remainder = deck[12:]

In [61]:
def hand_sum(hand):
    points = np.zeros(4)
    for elem in hand:
        if not None:
            points[elem[0]] += card_values[elem[1]]
    return np.max(points)

In [62]:
def turn(hand, open_cards):
    best_card = [hand_sum(hand), None, None]
    for card in open_cards:
        for position in range(3):
            test_hand = hand.copy()
            test_hand[position] = card
            handsum = hand_sum(test_hand)
            if handsum > best_card[0]:
                best_card[0], best_card[1], best_card[2] = handsum, open_cards.index(card), position
    if best_card[1] != None:
        hand[best_card[2]], open_cards[best_card[1]] = open_cards[best_card[1]], hand[best_card[2]]
    if hand_sum(hand) >= 31:
        return True
    return 

In [63]:
no_action = 0
players = [hand, oponent_Hand_1, oponent_hand_2]
window = tkinter.Tk()
window.title("FIVK")
window.configure(background="green")
window.attributes("-fullscreen", True)
window.attributes("-topmost", True)
label = []
for i in range(12):
    label.append(i)
    img = tkinter.PhotoImage(file= "PNG-cards-1.3\\48.png")
    label[i] = tkinter.Label(image=img)
    label[i].photo = img
    label[i].place(x = CorX[i], y = CorY[i])

def update():
    time.sleep(2)
    Hand = hand + open_cards
    cards = [] 
    for card in Hand:
        cards.append(f'PNG-cards-1.3\\{card[0]}{card[1]}.png')
    for j in range(6):
        print("K")
        img = tkinter.PhotoImage(file = cards[j])
        label[j].configure(image=img)
        label[j].photo = img
    return

for i in range(1000):
    if no_action == 3:
        print("Schiebung")
        open_cards, remainder[:3] = remainder[:3], open_cards
        shuffle(remainder)
        no_action = 0
    open_cards_before = open_cards.copy()
    print(players[i%3], i%3 + 1,  hand_sum(players[i%3]))
    print(open_cards, "Offenekarten")
    outcome = turn(players[i%3], open_cards)
    if outcome == True:
        print(f'Player {i%3 + 1} wins.')
        Hand = hand + open_cards + oponent_Hand_1 + oponent_hand_2
        cards = [] 
        for card in Hand:
            cards.append(f'PNG-cards-1.3\\{card[0]}{card[1]}.png')
        for k in range(12):
            img = tkinter.PhotoImage(file = cards[k])
            label[k].configure(image=img)
            label[k].photo = img
        break
    if open_cards == open_cards_before:
        no_action += 1
    update()


window.mainloop()


[(3, 3), (1, 4), (3, 6)] 1 20.0
[(0, 5), (1, 5), (1, 3)] Offenekarten
K
K
K
K
K
K
[(1, 7), (3, 2), (0, 0)] 2 11.0
[(0, 5), (1, 5), (1, 3)] Offenekarten
K
K
K
K
K
K
[(1, 0), (1, 1), (3, 7)] 3 15.0
[(0, 5), (3, 2), (1, 3)] Offenekarten
K
K
K
K
K
K
[(3, 3), (1, 4), (3, 6)] 1 20.0
[(0, 5), (3, 2), (3, 7)] Offenekarten
Player 1 wins.
